In [1]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

# 1. prepare

## 1-1 Setting and prepare transformation

In [4]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

# Creating the transformations
# We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) 


/anaconda3/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


## 1-2 Loading the dataset

In [5]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

Files already downloaded and verified


## 1-3 initialize weights

In [6]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# 2. Build model

## 2-1 build generator

In [7]:
# Defining the generator

class G(nn.Module): # We introduce a class to define the generator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # We start with an inversed convolution.
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the generator defined by self.main.
        return output # We return the output containing the generated images.

# Creating the generator
netG = G() # We create the generator object.
netG.apply(weights_init) # We initialize all the weights of its neural network.


G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

## 2-2 build discriminator

In [8]:
# Defining the discriminator

class D(nn.Module): # We introduce a class to define the discriminator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the discriminator.
        super(D, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), # We start with a convolution.
            nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), # We normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(512), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the discriminator defined by self.main.
        return output.view(-1) # We return the output which will be a value between 0 and 1.

# Creating the discriminator
netD = D() # We create the discriminator object.
netD.apply(weights_init) # We initialize all the weights of its neural network.



D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

# 3. Training the model (DCGANs)

## 3-1 Setting

In [10]:
criterion = nn.BCELoss() # We create a criterion object that will measure the error between the prediction and the target.
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the discriminator.
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the generator.

## 3-2 Train

In [21]:
for epoch in range(1): # We iterate over 25 epochs.

    for i, data in enumerate(dataloader, 0): # We iterate over the images of the dataset.
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad() # We initialize to 0 the gradients of the discriminator with respect to the weights.
        
        # Training the discriminator with a real image of the dataset
        real, _ = data # We get a real image of the dataset which will be used to train the discriminator.
        input = Variable(real) # We wrap it in a variable.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(input) # We forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_real = criterion(output, target) # We compute the loss between the predictions (output) and the target (equal to 1).
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # We make a random input vector (noise) of the generator.
        fake = netG(noise) # We forward propagate this random input vector into the neural network of the generator to get some fake generated images.
        target = Variable(torch.zeros(input.size()[0])) # We get the target.
        output = netD(fake.detach()) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_fake = criterion(output, target) # We compute the loss between the prediction (output) and the target (equal to 0).

        # Backpropagating the total error
        errD = errD_real + errD_fake # We compute the total error of the discriminator.
        errD.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
        optimizerD.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad() # We initialize to 0 the gradients of the generator with respect to the weights.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(fake) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errG = criterion(output, target) # We compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
        errG.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
        optimizerG.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.

        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item())) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True) # We also save the fake generated images of the minibatch.

[0/25][0/782] Loss_D: 0.0035 Loss_G: 34.8897
[0/25][1/782] Loss_D: 0.0019 Loss_G: 35.0996
[0/25][2/782] Loss_D: 0.0034 Loss_G: 35.1066
[0/25][3/782] Loss_D: 0.0046 Loss_G: 35.4700
[0/25][4/782] Loss_D: 0.0042 Loss_G: 34.9679
[0/25][5/782] Loss_D: 0.0039 Loss_G: 34.8763
[0/25][6/782] Loss_D: 0.0058 Loss_G: 34.5741
[0/25][7/782] Loss_D: 0.0040 Loss_G: 34.9144
[0/25][8/782] Loss_D: 0.0021 Loss_G: 34.7208
[0/25][9/782] Loss_D: 0.0123 Loss_G: 34.7685
[0/25][10/782] Loss_D: 0.0030 Loss_G: 34.7548
[0/25][11/782] Loss_D: 0.0057 Loss_G: 34.8664
[0/25][12/782] Loss_D: 0.0034 Loss_G: 34.5940
[0/25][13/782] Loss_D: 0.0045 Loss_G: 34.5235
[0/25][14/782] Loss_D: 0.0055 Loss_G: 34.3898
[0/25][15/782] Loss_D: 0.0030 Loss_G: 34.1745
[0/25][16/782] Loss_D: 0.0025 Loss_G: 34.2257
[0/25][17/782] Loss_D: 0.0026 Loss_G: 33.7302
[0/25][18/782] Loss_D: 0.0034 Loss_G: 34.0246
[0/25][19/782] Loss_D: 0.0030 Loss_G: 33.6545
[0/25][20/782] Loss_D: 0.0029 Loss_G: 33.5571
[0/25][21/782] Loss_D: 0.0049 Loss_G: 33.128

[0/25][180/782] Loss_D: 0.1821 Loss_G: 6.9652
[0/25][181/782] Loss_D: 0.1259 Loss_G: 5.9860
[0/25][182/782] Loss_D: 0.1246 Loss_G: 4.8215
[0/25][183/782] Loss_D: 0.1688 Loss_G: 6.5439
[0/25][184/782] Loss_D: 0.2295 Loss_G: 4.9326
[0/25][185/782] Loss_D: 0.1443 Loss_G: 5.8251
[0/25][186/782] Loss_D: 0.1943 Loss_G: 5.9231
[0/25][187/782] Loss_D: 0.1189 Loss_G: 5.7636
[0/25][188/782] Loss_D: 0.1510 Loss_G: 6.1988
[0/25][189/782] Loss_D: 0.1848 Loss_G: 6.5610
[0/25][190/782] Loss_D: 0.1516 Loss_G: 7.0023
[0/25][191/782] Loss_D: 0.1643 Loss_G: 6.5094
[0/25][192/782] Loss_D: 0.3877 Loss_G: 8.0236
[0/25][193/782] Loss_D: 0.1110 Loss_G: 7.2601
[0/25][194/782] Loss_D: 0.2742 Loss_G: 5.7898
[0/25][195/782] Loss_D: 0.3031 Loss_G: 7.5616
[0/25][196/782] Loss_D: 0.1772 Loss_G: 6.2356
[0/25][197/782] Loss_D: 0.1888 Loss_G: 7.9196
[0/25][198/782] Loss_D: 0.1273 Loss_G: 6.4003
[0/25][199/782] Loss_D: 0.1334 Loss_G: 7.4749
[0/25][200/782] Loss_D: 0.2360 Loss_G: 5.5075
[0/25][201/782] Loss_D: 0.1840 Los

[0/25][358/782] Loss_D: 0.5556 Loss_G: 4.3824
[0/25][359/782] Loss_D: 2.0417 Loss_G: 9.6066
[0/25][360/782] Loss_D: 0.9149 Loss_G: 7.4864
[0/25][361/782] Loss_D: 0.3580 Loss_G: 4.9417
[0/25][362/782] Loss_D: 0.6368 Loss_G: 6.3901
[0/25][363/782] Loss_D: 0.4609 Loss_G: 5.8274
[0/25][364/782] Loss_D: 0.8825 Loss_G: 4.7083
[0/25][365/782] Loss_D: 1.0049 Loss_G: 6.4771
[0/25][366/782] Loss_D: 0.8574 Loss_G: 4.1116
[0/25][367/782] Loss_D: 1.0411 Loss_G: 6.5740
[0/25][368/782] Loss_D: 0.2414 Loss_G: 6.2645
[0/25][369/782] Loss_D: 0.3477 Loss_G: 5.0168
[0/25][370/782] Loss_D: 0.3115 Loss_G: 4.5391
[0/25][371/782] Loss_D: 0.6425 Loss_G: 3.6376
[0/25][372/782] Loss_D: 0.6644 Loss_G: 4.4375
[0/25][373/782] Loss_D: 0.6241 Loss_G: 4.9519
[0/25][374/782] Loss_D: 0.5161 Loss_G: 4.5157
[0/25][375/782] Loss_D: 0.2800 Loss_G: 3.9999
[0/25][376/782] Loss_D: 0.6000 Loss_G: 6.1756
[0/25][377/782] Loss_D: 0.6713 Loss_G: 2.5319
[0/25][378/782] Loss_D: 0.9721 Loss_G: 5.9434
[0/25][379/782] Loss_D: 0.9976 Los

[0/25][537/782] Loss_D: 0.3482 Loss_G: 4.2525
[0/25][538/782] Loss_D: 0.7982 Loss_G: 4.6138
[0/25][539/782] Loss_D: 0.3960 Loss_G: 4.6389
[0/25][540/782] Loss_D: 0.5026 Loss_G: 3.3169
[0/25][541/782] Loss_D: 0.8977 Loss_G: 7.5829
[0/25][542/782] Loss_D: 0.8228 Loss_G: 3.9651
[0/25][543/782] Loss_D: 0.7087 Loss_G: 4.9369
[0/25][544/782] Loss_D: 0.2834 Loss_G: 5.2243
[0/25][545/782] Loss_D: 0.2205 Loss_G: 4.4281
[0/25][546/782] Loss_D: 0.3612 Loss_G: 4.6537
[0/25][547/782] Loss_D: 0.3053 Loss_G: 3.6731
[0/25][548/782] Loss_D: 0.6726 Loss_G: 5.9310
[0/25][549/782] Loss_D: 0.6622 Loss_G: 2.0879
[0/25][550/782] Loss_D: 1.5061 Loss_G: 11.0822
[0/25][551/782] Loss_D: 2.3246 Loss_G: 5.7593
[0/25][552/782] Loss_D: 0.2988 Loss_G: 2.5469
[0/25][553/782] Loss_D: 1.3170 Loss_G: 8.0653
[0/25][554/782] Loss_D: 1.5270 Loss_G: 3.1680
[0/25][555/782] Loss_D: 1.0113 Loss_G: 4.0918
[0/25][556/782] Loss_D: 0.8373 Loss_G: 3.7757
[0/25][557/782] Loss_D: 0.7279 Loss_G: 4.9326
[0/25][558/782] Loss_D: 1.0971 Lo

[0/25][716/782] Loss_D: 0.6036 Loss_G: 5.4812
[0/25][717/782] Loss_D: 0.3767 Loss_G: 4.8490
[0/25][718/782] Loss_D: 0.3772 Loss_G: 4.6572
[0/25][719/782] Loss_D: 0.2629 Loss_G: 5.0160
[0/25][720/782] Loss_D: 0.2936 Loss_G: 5.3786
[0/25][721/782] Loss_D: 0.5308 Loss_G: 3.4389
[0/25][722/782] Loss_D: 0.8192 Loss_G: 7.7601
[0/25][723/782] Loss_D: 1.0202 Loss_G: 3.1165
[0/25][724/782] Loss_D: 0.9392 Loss_G: 7.4238
[0/25][725/782] Loss_D: 0.6265 Loss_G: 4.0182
[0/25][726/782] Loss_D: 0.5746 Loss_G: 5.1075
[0/25][727/782] Loss_D: 0.6791 Loss_G: 3.7579
[0/25][728/782] Loss_D: 0.8760 Loss_G: 5.5110
[0/25][729/782] Loss_D: 0.5532 Loss_G: 3.8451
[0/25][730/782] Loss_D: 0.4146 Loss_G: 4.2394
[0/25][731/782] Loss_D: 0.2692 Loss_G: 5.3350
[0/25][732/782] Loss_D: 0.2852 Loss_G: 4.2246
[0/25][733/782] Loss_D: 0.2923 Loss_G: 4.5628
[0/25][734/782] Loss_D: 0.6999 Loss_G: 2.9415
[0/25][735/782] Loss_D: 0.6004 Loss_G: 6.3733
[0/25][736/782] Loss_D: 0.4279 Loss_G: 3.6214
[0/25][737/782] Loss_D: 0.3899 Los